In [77]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

# Загрузка данных

In [78]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [79]:
data = pd.read_excel('Data/data_ford_price.xlsx') 

#  Отбор признаков: мотивация

## Предобработка данных

In [80]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

## Обучение модели

In [81]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = metrics.mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4682.957


## Удаление избыточного признака

In [82]:
# Мы выяснили, что у нас присутствует сильная зависимость между lat и weather (тепловая карта).
# Удалим lat, так как этот признак, в отличие от weather, необходимо округлять.
x.drop('lat', axis = 1, inplace = True)

In [83]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [84]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = metrics.mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4672.930


#  Отбор признаков: классификация методов

## Метод рекурсивного исключения признаков

In [85]:
from sklearn.feature_selection import RFE

In [86]:
y = data['price']
x = data.drop(columns='price')

In [87]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

Метод рекурсивного исключения признаков (RFE) предполагает выбор признаков путём рекурсивного рассмотрения всё меньших и меньших наборов фичей.

Сначала RFE обучается на изначальной выборке и происходит оценка важности каждого признака. Затем наименее важные фичи удаляются. Эта процедура рекурсивно повторяется на сокращённом наборе до тех пор, пока в конечном итоге не будет достигнуто желаемое количество признаков в выборке.

Выделим три наиболее значимых признака:

In [88]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [89]:
X_train.columns

Index(['year', 'cylinders', 'odometer', 'lat', 'long', 'weather'], dtype='object')

In [90]:
selector.ranking_

array([1, 1, 4, 1, 3, 2])

Признаку odometer соответствует наибольшее число (4), следовательно, данный признак наименее значимый.

##  МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ

Библиотека sklearn обеспечивает реализацию большинства полезных статистических показателей, например:

        коэффициента корреляции Пирсона: f_regression();
        дисперсионного анализа ANOVA: f_classif();
        хи-квадрата: chi2();
        взаимной информации: mutual_info_classif() и mutual_info_regression().

Кроме того, библиотека SciPy обеспечивает реализацию многих других статистических данных, таких как тау Кендалла (kendalltau) и ранговая корреляция Спирмена (spearmanr).

sklearn также предоставляет множество различных методов фильтрации после расчёта статистики для каждой входной переменной с целевой.

Два наиболее популярных метода:

        выбор k лучших переменных: SelectKBest;
        выбор переменных верхнего процентиля: SelectPercentile.


In [91]:
from sklearn.feature_selection import SelectKBest, f_regression

In [92]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

На этот раз odometer оказался в топе.

### Задание 9.5. Модуль ML-6 (HW-03)
Обучите модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты. 

- Верно выделены три столбца-признака для обучения, выбранные RFE.
- Верно выделены три столбца-признака для обучения, выбранные SelectKBest.
- Обучена регрессия на первых трёх столбцах, оценено качество модели на тесте.
- Обучена регрессия на вторых трёх столбцах, оценено качество модели на тесте.
- Произведено сравнение выбранных метрик в форме комментария. Дан ответ на вопрос «Какой метод отбора признаков показал наилучший результат на тестовой выборке?» (в текстовой ячейке).

Приоритетные признаки:
- рекурсивное исключение признаков: 'year', 'cylinders', 'lat'
- выбор на основе фильтров: 'year', 'cylinders', 'odometer'

In [93]:
# Пропишем функцию для выведения метрик моделей
def print_metrics(y_test, y_predicted):
    print('Test R^2: {:.3f}'.format(metrics.r2_score(y_test, y_predicted)))
    print('Test MAE: {:.3f}'.format(metrics.mean_absolute_error(y_test, y_predicted)))
    print('Test MAPE: {:.3f}'.format(metrics.mean_absolute_percentage_error(y_test, y_predicted)*100))

In [94]:
# Выведем справочно метрики первонально построенной модели
print('Первоначальная модель (справочно):')
print_metrics(y_test, y_predicted)

Первоначальная модель (справочно):
Test R^2: 0.606
Test MAE: 4672.930
Test MAPE: 16769.685


In [95]:
# Начнем с первого набора признаков: 'year', 'cylinders', 'lat'

#Создаем матрицу наблюдений X и столбец с ответами y
X = data[['year', 'cylinders', 'lat']]
y = data['price']

# Разобьем выборки на тестовую и тренировочную выборки
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Создадим модель регрессии
model = LinearRegression()

# Обучим модель на тренировочной выборке
model.fit(X_train, y_train)

# Предскажем целевой признак для тестовой выборки
y_predicted = model.predict(X_test)

In [96]:
# Отразим значения метрик для модели на первом наборе признаков
print('RFE:')
print_metrics(y_test, y_predicted)

RFE:
Test R^2: 0.605
Test MAE: 4863.525
Test MAPE: 15784.967


In [97]:
# Второй набор признаков: 'year', 'cylinders', 'odometer'

#Создаем матрицу наблюдений X и столбец с ответами y
X = data[['year', 'cylinders', 'odometer']]
y = data['price']

# Разобьем выборки на тестовую и тренировочную выборки
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Создадим модель регрессии
model = LinearRegression()

# Обучим модель на тренировочной выборке
model.fit(X_train, y_train)

# Предскажем целевой признак для тестовой выборки
y_predicted = model.predict(X_test)

In [98]:
# Отразим значения метрик для модели на втором наборе признаков
print('SelectKBest:')
print_metrics(y_test, y_predicted)

SelectKBest:
Test R^2: 0.605
Test MAE: 4863.525
Test MAPE: 15784.967


### Какой метод отбора признаков показал наилучший результат на тестовой выборке?
Модели на основании обоих наборов факторов показали одинаковые метрики - они одинаково справляются с тестовой выборкой.